In [2]:
import io
import os
import re
import keras
import tensorflow as tf
import numpy as np # used for handling numbers
import pandas as pd # used for handling the dataset
df= pd.read_csv('drive/My Drive/Colab Notebooks/dataset/labeled_outs.txt',delimiter=",",header=None)
kw=pd.read_csv('drive/My Drive/Colab Notebooks/dataset/frequent.txt',header=None)
# Dataset is now stored in a Pandas Dataframe

Using TensorFlow backend.


In [0]:
df.columns=['label','text']
df.text=df.text.astype(str)
df.label=df.label.astype(str)
df=df.sample(frac=1)
docs=df['text'].values
labels=df['label'].values

In [0]:

from sklearn.feature_extraction.text import CountVectorizer
input_dim=5000
count_vect = CountVectorizer(max_features = input_dim)
X_train_counts = count_vect.fit_transform(docs)

In [0]:
from sklearn.feature_extraction.text import TfidfTransformer

tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
X_train_tfidf.shape
X_train_tfidf = X_train_tfidf.toarray()
X_train_tfidf = np.reshape(X_train_tfidf, (X_train_tfidf.shape[0],10, -1))
print(X_train_tfidf.shape)

(91795, 1, 5000)


In [4]:

kw.columns=['words']
dfreq={}
for k in kw['words']:
  dfreq[k]=0
print(len(dfreq))

9000


In [5]:
for d in docs:
  words=re.split('[\s]',d)
  words=np.unique(words)
  for w in words:
    if w in dfreq:
      dfreq[w]=dfreq[w]+1

tf_idf = []
for d in docs:
  words=re.split('[\s]',d)
  N=len(words)
  tfreq={}
  for w in words:
    if w in tfreq:
      tfreq[w]=tfreq[w]+1
    else:
      tfreq[w]=1
  temp=[]
  for w in dfreq:
    if w in tfreq:
      tf=tfreq[w]*1.0/N
      idf=np.log(len(docs)*1.0/dfreq[w])
      temp.append(tf*idf)
    else:
      temp.append(0.0)
  tf_idf.append(temp)

tf_idf = np.array(tf_idf)
input_dim=len(tf_idf[0])
print(input_dim)
tf_idf = np.reshape(tf_idf, (tf_idf.shape[0],1,-1))

9000


In [6]:

from keras.utils import np_utils
from sklearn.preprocessing import LabelEncoder
print(labels)
# encode class values as integers
encoder = LabelEncoder()
encoder.fit(labels)
encoded_Y = encoder.transform(labels)
# convert integers to dummy variables (i.e. one hot encoded)
dummy_y = np_utils.to_categorical(encoded_Y)
print(dummy_y)


['sports' 'international' 'science_tech' ... 'sports' 'entertainment'
 'sports']
[[0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 1. 0. 0.]
 ...
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 1. 0.]]


In [0]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(tf_idf, dummy_y, test_size=0.18)

In [8]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)


(75271, 1, 9000)
(75271, 8)
(16524, 1, 9000)
(16524, 8)


In [0]:
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout

model = Sequential()
model.add(LSTM(2048, input_shape=(X_train.shape[1],X_train.shape[2]),return_sequences=True,dropout=0.5,recurrent_dropout=0.3))
model.add(LSTM(1024,return_sequences=True,dropout=0.4,recurrent_dropout=0.3))
model.add(LSTM(512,return_sequences=True,dropout=0.4,recurrent_dropout=0.3))
model.add(LSTM(256,return_sequences=True,dropout=0.3,recurrent_dropout=0.2))
model.add(LSTM(128,return_sequences=True,dropout=0.3,recurrent_dropout=0.2))
model.add(LSTM(64,return_sequences=True,dropout=0.2,recurrent_dropout=0.1))
model.add(LSTM(32,return_sequences=True))
model.add(LSTM(16))
model.add(Dense(8, activation='softmax'))

In [10]:

model.compile(loss="categorical_crossentropy",optimizer="adam",metrics=['acc'])
# summarize the model
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 1, 2048)           90513408  
_________________________________________________________________
lstm_2 (LSTM)                (None, 1, 1024)           12587008  
_________________________________________________________________
lstm_3 (LSTM)                (None, 1, 512)            3147776   
_________________________________________________________________
lstm_4 (LSTM)                (None, 1, 256)            787456    
_________________________________________________________________
lstm_5 (LSTM)                (None, 1, 128)            197120    
_________________________________________________________________
lstm_6 (LSTM)                (None, 1, 64)             49408     
_________________________________________________________________
lstm_7 (LSTM)                (None, 1, 32)            

In [11]:
history = model.fit(X_train, y_train, epochs=30,batch_size=256,shuffle=False, verbose=1,validation_split=0.1)
#model.save_weights('./checkpoints/text_classifier_rnn_tfidf')
!mkdir -p saved_model
model.save('saved_model/text_classifier_rnn_tfidf')

Train on 67743 samples, validate on 7528 samples
Epoch 1/30
67743/67743 [==============================] - 29s 421us/step - loss: 1.7997 - acc: 0.2222 - val_loss: 1.5036 - val_acc: 0.2984
Epoch 2/30
67743/67743 [==============================] - 23s 345us/step - loss: 1.2492 - acc: 0.4972 - val_loss: 0.7639 - val_acc: 0.7228
Epoch 3/30
67743/67743 [==============================] - 23s 345us/step - loss: 0.6557 - acc: 0.7894 - val_loss: 0.4183 - val_acc: 0.8535
Epoch 4/30
67743/67743 [==============================] - 23s 344us/step - loss: 0.4502 - acc: 0.8560 - val_loss: 0.3019 - val_acc: 0.9261
Epoch 5/30
67743/67743 [==============================] - 23s 346us/step - loss: 0.3237 - acc: 0.9136 - val_loss: 0.2380 - val_acc: 0.9416
Epoch 6/30
67743/67743 [==============================] - 23s 344us/step - loss: 0.2770 - acc: 0.9243 - val_loss: 0.2190 - val_acc: 0.9398
Epoch 7/30
67743/67743 [==============================] - 23s 345us/step - loss: 0.2530 - acc: 0.9295 - val_loss: 0.2

In [0]:
#model.load_weights('./checkpoints/text_classifier_rnn_tfidf')
new_model = tf.keras.models.load_model('saved_model/text_classifier_rnn_tfidf')

In [12]:
results = model.evaluate(X_test, y_test)
print(results)

16524/16524 [==============================] - 3s 181us/step
[0.18679943855890666, 0.9472888112068176]


In [0]:
y_pred=model.predict(X_test)

In [14]:
"""
import numpy as np
array1=[[ 0,  1,  2,  3,  4],
       [ 0,  1,  2,  3,  4],
       [5, 6, 7, 1, 0],
       [5, 6, 7, 1, 0]]
array2=[[ 5, 6, 7, 1, 0],
       [ 0,  1,  2,  3,  4],
       [0,  1,  2,  3,  4],
       [5, 6, 7, 1, 0]]
"""
no_cat=len(y_test[0])
conf_mat=np.zeros(no_cat*no_cat).reshape(no_cat,no_cat)
test_size=len(y_test)
print(conf_mat.shape)
for i in range(test_size):
  y_t=y_test[i]
  y_p=y_pred[i]
  class_t=np.where(y_t == np.amax(y_t))
  class_t=class_t[0][0]
  class_p=np.where(y_p == np.amax(y_p))
  class_p=class_p[0][0]
  conf_mat[class_t][class_p]=conf_mat[class_t][class_p]+1

(8, 8)


In [15]:
tp=np.zeros(no_cat)
fp=np.zeros(no_cat)
tn=np.zeros(no_cat)
fn=np.zeros(no_cat)

prec=np.zeros(no_cat)
rec=np.zeros(no_cat)
f_score=np.zeros(no_cat)

for i in range(no_cat):
  for j in range(no_cat):
    if i==j:
      tp[i]=conf_mat[i][j]
    if i!= j:
      fp[i] = fp[i]+conf_mat[j][i]
      fn[i] = fn[i]+conf_mat[i][j]
    for k in range(no_cat):
      if i!=k & j!=k:
        tn[i]=tn[i]+conf_mat[j][k]

for i in range(no_cat):
    prec[i] = tp[i] * 100.0 / (tp[i] + fp[i])
    rec[i] = tp[i] * 100.0 / (tp[i] + fn[i])
    f_score[i] = 2 * prec[i] * rec[i] / (prec[i] + rec[i])

print(np.average(prec))
print(np.average(rec))
print(np.average(f_score))

94.70550501489832
94.7150071671608
94.70272489196634
